In [1]:
import numpy as np
import pandas as pd
import cv2, os
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from sklearn.model_selection import train_test_split

# Import classifiers and performance metrics
from sklearn import svm, metrics

In [33]:
# Read dataset

birds_dir = os.listdir('../dataset/birds/')
drones_dir = os.listdir('../dataset/drones/')
birds = []
drones = []
for i in range(800):
    print(i, drones_dir[i])
    birds.append(cv2.resize(cv2.imread("../dataset/birds/" + birds_dir[i], cv2.IMREAD_GRAYSCALE), (200, 200)).flatten())
    drones.append(cv2.resize(cv2.imread("../dataset/drones/" + drones_dir[i], cv2.IMREAD_GRAYSCALE), (200, 200)).flatten())

0 3_images (4).png
1 dji_phantom_3_download (15).jpeg
2 1_images (80).jpeg
3 3_images (50).jpeg
4 dji_mavic_images (20).jpeg
5 3_images (15).jpeg
6 dji_mavic_images (65).jpeg
7 dji_mavic_images (98).jpeg
8 1_images (38).jpeg
9 stalker_images (63).jpeg
10 stalker_images (26).jpeg
11 dji_phantom_3_download (3).jpeg
12 3_download (6).jpeg
13 2_images (93).jpeg
14 dji_phantom_3_images (69).jpeg
15 stalker_download (5).jpeg
16 2_images (5).png
17 dji_phantom_3_images (54).jpeg
18 dji_phantom_3_images (11).jpeg
19 dji_phantom_3_images (2).jpeg
20 2_images (16).jpeg
21 2_images (53).jpeg
22 1_images (40).jpeg
23 3_images (90).jpeg
24 1_download (12).jpeg
25 1_images (18).png
26 dji_mavic_images (58).jpeg
27 3_images (28).jpeg
28 2_images (32).jpeg
29 2_images (77).jpeg
30 1_download (5).jpeg
31 2_images (100).jpeg
32 2_images (8).jpeg
33 3_download (25).jpeg
34 dji_phantom_3_images (70).jpeg
35 dji_phantom_3_images (35).jpeg
36 2_fb_32x32.png
37 3_images (49).jpeg
38 dji_mavic_images (39).jpe

445 dji_mavic_download (16).jpeg
446 3_images (6).jpeg
447 1_images (76).jpeg
448 1_images (33).jpeg
449 dji_mavic_images (93).jpeg
450 stalker_images (68).jpeg
451 dji_mavic_images (3).png
452 dji_phantom_3_download (8).jpeg
453 2_download.jpeg
454 2_images (20).jpeg
455 2_images (65).jpeg
456 2_images (98).jpeg
457 dji_mavic_images (6).jpeg
458 3_images (12).png
459 dji_phantom_3_images (62).jpeg
460 stalker_fb_32x32.png
461 dji_phantom_3_images (27).jpeg
462 stalker_images (4).jpeg
463 1_images (74).jpeg
464 1_images (31).jpeg
465 dji_mavic_images (91).jpeg
466 dji_mavic_images (29).jpeg
467 3_images (59).jpeg
468 dji_mavic_download (14).jpeg
469 1_images (89).jpeg
470 3_images (4).jpeg
471 1_download (26).jpeg
472 dji_phantom_3_images (60).jpeg
473 dji_phantom_3_images (25).jpeg
474 stalker_images (6).jpeg
475 1_images (26).png
476 dji_mavic_images (4).jpeg
477 2_images (22).jpeg
478 2_images (67).jpeg
479 dji_phantom_3_gplus_32x32.png
480 dji_phantom_3_images (18).jpeg
481 3_image

In [38]:
img_data = np.concatenate((np.array(birds), np.array(drones)), axis=0)
img_target = []
for i in range(1600):
    if i<800:
        img_target.append(0)
    else:
        img_target.append(1)
img_target = np.array(img_target)

In [39]:
X_train, X_test, y_train, y_test = train_test_split(img_data, img_target)

In [40]:
# Create a classifier: a support vector classifier
classifier = svm.SVC(gamma=0.001, C=9)
#fit to the trainin data
classifier.fit(X_train,y_train)

SVC(C=9, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [41]:
y_pred = classifier.predict(X_test)

In [42]:
print("Classification report for classifier %s:\n%s\n"
      % (classifier, metrics.classification_report(y_test, y_pred)))

Classification report for classifier SVC(C=9, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False):
              precision    recall  f1-score   support

           0       0.60      0.03      0.05       214
           1       0.47      0.98      0.63       186

   micro avg       0.47      0.47      0.47       400
   macro avg       0.53      0.50      0.34       400
weighted avg       0.54      0.47      0.32       400




In [43]:
print("Confusion matrix:\n%s" % metrics.confusion_matrix(y_test, y_pred))

Confusion matrix:
[[  6 208]
 [  4 182]]
